In [1]:
from env import host, user, password
import acquire
import prepare
import wrangle

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
df = acquire.get_zillow_file()

In [4]:
zillow= prepare.prep_zillow(df)

In [5]:
zillow.head()

,total_sqft,bedrooms,bathrooms,value_assessed,tax_amount,year_built,county_code,parcelid,tax_rate
0,1316.0,3.0,2.0,205123.0,2627.48,1923.0,6037.0,11721753,1.280929
1,1458.0,3.0,2.0,136104.0,2319.90,1970.0,6037.0,11289917,1.704505
2,1421.0,2.0,1.0,35606.0,543.69,1911.0,6037.0,11705026,1.526962
3,2541.0,4.0,3.0,880456.0,9819.72,2003.0,6059.0,14269464,1.115299
4,1491.0,3.0,2.0,107110.0,1399.27,1955.0,6037.0,11446756,1.306386


In [6]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28292 entries, 0 to 28417
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   total_sqft      28292 non-null  float64
 1   bedrooms        28292 non-null  float64
 2   bathrooms       28292 non-null  float64
 3   value_assessed  28292 non-null  float64
 4   tax_amount      28292 non-null  float64
 5   year_built      28292 non-null  float64
 6   county_code     28292 non-null  float64
 7   parcelid        28292 non-null  int64  
 8   tax_rate        28292 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 2.2 MB


In [7]:
lowerbound, upperbound = prepare.outlier_bound_calculation(zillow, 'bedrooms')

For bedrooms the lower bound is 1.5 and  upper bound is 5.5


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
#split 
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.train_validate_test(zillow, 'taxvaluedollarcnt')

In [ ]:
#add numeric columns to be scaled (except y_train (taxvaluedollarcnt))
numeric_cols = ['calculatedfinishedsquarefeet', 'bedroomcnt', 'bathroomcnt',
       'taxamount', 'unitcnt', 'yearbuilt', 'fips', 'tax_rate']

In [ ]:
#scale data
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.min_max_scale(X_train, X_validate, X_test, numeric_cols)